<a href="https://colab.research.google.com/github/kuroro-31/LSTM-v1/blob/master/LSTM_99_5_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 必要なライブラリのインポート
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
import datetime

today = datetime.date.today()

In [ ]:
# データの取得
data = yf.download('8058.T', start='2010-01-01', end=today)

# クロージング価格のみを使用
close_price = data.iloc[:, 3:4].values

# データの正規化
scaler = MinMaxScaler()
close_price_scaled = scaler.fit_transform(close_price)

# 過去60日分のデータを使用して次の日の株価を予測
X_train = []
y_train = []

for i in range(60, len(close_price)):
    X_train.append(close_price_scaled[i-60:i, 0])
    y_train.append(close_price_scaled[i, 0])
    
X_train, y_train = np.array(X_train), np.array(y_train)

# LSTMへの入力に適した形に変形
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# LSTMネットワークの作成
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))

[*********************100%***********************]  1 of 1 completed


In [ ]:
# モデルのコンパイルと学習
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
51/51 [==============================] - 14s 9ms/step - loss: 0.0135
Epoch 2/50
51/51 [==============================] - 0s 7ms/step - loss: 0.0019
Epoch 3/50
51/51 [==============================] - 0s 8ms/step - loss: 0.0018
Epoch 4/50
51/51 [==============================] - 0s 7ms/step - loss: 0.0016
Epoch 5/50
51/51 [==============================] - 0s 8ms/step - loss: 0.0016
Epoch 6/50
51/51 [==============================] - 0s 8ms/step - loss: 0.0014
Epoch 7/50
51/51 [==============================] - 0s 8ms/step - loss: 0.0014
Epoch 8/50
51/51 [==============================] - 0s 8ms/step - loss: 0.0014
Epoch 9/50
51/51 [==============================] - 0s 8ms/step - loss: 0.0013
Epoch 10/50
51/51 [==============================] - 0s 8ms/step - loss: 0.0013
Epoch 11/50
51/51 [==============================] - 0s 8ms/step - loss: 0.0012
Epoch 12/50
51/51 [==============================] - 0s 8ms/step - loss: 0.0012
Epoch 13/50
51/51 [=============================

In [ ]:
# モデルの予測結果をテキストとして表示（最新の10日間のみ）
for i in range(len(predicted_stock_price)-10, len(predicted_stock_price)):
    print(f"Date: {data.index[len(data)-len(predicted_stock_price)+i]} - Actual price {close_price[len(data)-len(predicted_stock_price)+i][0]}, Predicted price {predicted_stock_price[i][0]}")

In [ ]:
import pandas as pd
import holidays

# 日本の祝日のリストを取得
jp_holidays = holidays.Japan(years=list(range(2000, 2025))).keys()
jp_holidays = [pd.to_datetime(date) for date in jp_holidays]

# カスタムビジネスデイカレンダーを作成
custom_business_day = pd.offsets.CustomBusinessDay(holidays=jp_holidays)

# 次の営業日を取得
next_business_day = pd.to_datetime(today) + custom_business_day

# 最新の60日間のデータを用いて次の日の株価を予測
last_60_days = close_price_scaled[-60:]
last_60_days = np.reshape(last_60_days, (1, 60, 1))
next_day_price = model.predict(last_60_days)
next_day_price = scaler.inverse_transform(next_day_price)

print(f"Predicted price for {next_business_day.strftime('%Y-%m-%d')}: {next_day_price[0][0]}")

# モデルの精度（整合率）の計算
mse = np.mean((real_stock_price.flatten() - predicted_stock_price.flatten()) ** 2)
accuracy = 1 - mse / np.var(real_stock_price.flatten())

print(f'モデル精度: {accuracy * 100:.2f}%')